# Dataset Exploration, Discovery and Learnings

All datasets were explored using Pandas in this notebook. 

I wanted to get a grasp on the 'gotchas' with the data transformations
needed to complete our schema. 

Additionally, the building of the schema tables were tested in this notebook. 

In [1]:
import pandas as pd
import zipfile
from zipfile import ZipFile
import json
import os
import glob
import numpy as np

# 1. SSN Birth Name Data
https://www.ssa.gov/oact/babynames/limits.html

6M+ records...  
- read multiple txt files into a single dataframe  
- dataset contains no headers; set column headers  
- add new columns for decade start and end  
- set a unique id

In [2]:
# loop through all state baby name text files

path = "/home/workspace/source_namesbystate/"
files = glob.glob(path + "*.TXT")

namesbystate_dfs = []
for f in files:
    df = namesbystate_dfs.append(pd.read_csv
                    (f, header=None))
    
namesbystate_dfs[1].head(5)

,0,1,2,3,4
0,CT,F,1910,Mary,304
1,CT,F,1910,Helen,170
2,CT,F,1910,Anna,131
3,CT,F,1910,Margaret,99
4,CT,F,1910,Dorothy,90


In [3]:
# transform into a single dataframe with headers

namesbystate_df =  pd.concat(namesbystate_dfs, 
                       sort=True, 
                       ignore_index=True, 
                       axis=0).rename({0: 'birth_state_code', 
                                       1: 'gender',
                                       2: 'birth_year',
                                       3: 'birth_name', 
                                       4:'popularity'}, axis=1) 

namesbystate_df['birth_name'] = namesbystate_df['birth_name'].str.lower()

namesbystate_df.head(25)

,birth_state_code,gender,birth_year,birth_name,popularity
0,NY,F,1910,mary,1923
1,NY,F,1910,helen,1290
2,NY,F,1910,rose,990
3,NY,F,1910,anna,951
4,NY,F,1910,margaret,926
5,NY,F,1910,dorothy,897
6,NY,F,1910,ruth,713
7,NY,F,1910,lillian,648
8,NY,F,1910,florence,604
9,NY,F,1910,frances,589


In [4]:
# add fields to identify records by decade
# for row in namesbystate_df:
#     namesbystate_df['decade_start'] = (namesbystate_df['birth_year']//10*10)
#     namesbystate_df['decade_end'] = ((namesbystate_df['birth_year']//10*10)+9)
    
# namesbystate_df['decade_start'] = np.round(namesbystate_df['decade_start'], decimals=0)
# namesbystate_df['decade_end'] = np.round(namesbystate_df['decade_end'], decimals=0)

In [5]:
# set a unique id (use the index)

namesbystate_df['babyName_id'] = namesbystate_df.index

In [6]:
# reorder the dataframe

namesbystate_df = namesbystate_df[[
                'babyName_id', 
                'birth_year',
#                 'decade_start', 
#                 'decade_end',
                'birth_name',
                'popularity',
                'birth_state_code',
                'gender']]

namesbystate_df.head(2)

,babyName_id,birth_year,birth_name,popularity,birth_state_code,gender
0,0,1910,mary,1923,NY,F
1,1,1910,helen,1290,NY,F


In [7]:
namesbystate_df.drop_duplicates().describe()

,babyName_id,birth_year,popularity
count,6.311504e+06,6.311504e+06,6.311504e+06
mean,3.155752e+06,1.977182e+03,5.065856e+01
std,1.821974e+06,3.127894e+01,1.731193e+02
min,0.000000e+00,1.910000e+03,5.000000e+00
25%,1.577876e+06,1.953000e+03,7.000000e+00
50%,3.155752e+06,1.983000e+03,1.200000e+01
75%,4.733627e+06,2.005000e+03,3.300000e+01
max,6.311503e+06,2.021000e+03,1.002600e+04


# 2. Storm Data

HURDAT2 description: https://www.aoml.noaa.gov/hrd/hurdat/Data_Storm.html \
HURDAT2 legend: https://www.aoml.noaa.gov/hrd/hurdat/hurdat2-format.pdf \
HURDAT2 dataset: https://www.nhc.noaa.gov/data/hurdat/hurdat2-1851-2021-041922.txt

55K+ records reduced to ~26K...    
- set headers
- drop unneeded columns
- data comes in as a multiline txt...
    - level 0 = storm id, storm name and count of the storm metadata records that follow
    - level 1 = storm metadata records
- 'flattened' the dataset (where storm id and storm name must be matched to n number of metadata records)
- filter the dataset by named storms
- decode a subset of fields (parse storm id, record identifier and storm type)
- converted time to numeric
- using the Saffir-Simpson Wind Scale categories as reference, map numeric categories to named storms
- strip coordinates of characters
- map coordinates to state code using geopy and an open source map API, where the named storm made landfall
- build tables
- validate

In [8]:
## one time read source data into Udacity workspace

# import urllib.request 
# urllib.request.urlretrieve('https://www.nhc.noaa.gov/data/hurdat/hurdat2-1851-2021-041922.txt', '/home/workspace/source_weather/hurdat2-1851-2021-041922.txt')

In [9]:
# read file ('https://www.nhc.noaa.gov/data/hurdat/hurdat2-1851-2021-041922.txt')
# there is no header
# name columns

import pandas as pd
HURDAT2 = pd.read_csv (r'/home/workspace/source_weather/hurdat2-1851-2021-041922.txt', header=None,
                  names=['storm_id',
                        'storm_name',
                        'associated_records',
                        'storm_date',
                        'storm_time',
                        'rec_identifier',
                        'storm_type',
                        'latitude',
                        'longitude',
                        'max_sustained_wind(kt)',
                        'minimum_pressure(mbar)',
                        "34_kt_wind_radii_maximum_extent_ne_quadrant_(in_nautical_miles)",
                        "34_kt_wind_radii_maximum_extent_se_quadrant_(in_nautical_miles)",
                        "34_kt_wind_radii_maximum_extent_sw_quadrant_(in_nautical_miles)",
                        "34_kt_wind_radii_maximum_extent_nw_quadrant_(in_nautical_miles)",
                        "50_kt_wind_radii_maximum_extent_ne_quadrant_(in_nautical_miles)",
                        "50_kt_wind_radii_maximum_extent_se_quadrant_(in_nautical_miles)",
                        "50_kt_wind_radii_maximum_extent_sw_quadrant_(in_nautical_miles)",
                        "50_kt_wind_radii_maximum_extent_nw_quadrant_(in_nautical_miles)",
                        "64_kt_wind_radii_maximum_extent_ne_quadrant_(in_nautical_miles)",
                        "64_kt_wind_radii_maximum_extent_se_quadrant_(in_nautical_miles)",
                        "64_kt_wind_radii_maximum_extent_sw_quadrant_(in_nautical_miles)",
                        "64_kt_wind_radii_maximum_extent_nw_quadrant_(in_nautical_miles)"
                        ])

In [10]:
print(HURDAT2.shape)
# HURDAT2.head(18)

(55437, 23)


## remove leading and trailing characters in headers

In [11]:
HURDAT2_obj = HURDAT2.select_dtypes(['object'])
HURDAT2[HURDAT2_obj.columns] = HURDAT2_obj.apply(lambda x: x.str.strip())

## remove unneeded columns

In [12]:
# check headers

HURDAT2.columns

Index(['storm_id', 'storm_name', 'associated_records', 'storm_date',
       'storm_time', 'rec_identifier', 'storm_type', 'latitude', 'longitude',
       'max_sustained_wind(kt)', 'minimum_pressure(mbar)',
       '34_kt_wind_radii_maximum_extent_ne_quadrant_(in_nautical_miles)',
       '34_kt_wind_radii_maximum_extent_se_quadrant_(in_nautical_miles)',
       '34_kt_wind_radii_maximum_extent_sw_quadrant_(in_nautical_miles)',
       '34_kt_wind_radii_maximum_extent_nw_quadrant_(in_nautical_miles)',
       '50_kt_wind_radii_maximum_extent_ne_quadrant_(in_nautical_miles)',
       '50_kt_wind_radii_maximum_extent_se_quadrant_(in_nautical_miles)',
       '50_kt_wind_radii_maximum_extent_sw_quadrant_(in_nautical_miles)',
       '50_kt_wind_radii_maximum_extent_nw_quadrant_(in_nautical_miles)',
       '64_kt_wind_radii_maximum_extent_ne_quadrant_(in_nautical_miles)',
       '64_kt_wind_radii_maximum_extent_se_quadrant_(in_nautical_miles)',
       '64_kt_wind_radii_maximum_extent_sw_quadrant_(i

In [13]:
"""
The raw data is multiline... 
level0 data aligns under columns 0-2.
level1 data aligns under columns 3-10.

For this project, not all columns are needed.
Drop the columns that don't apply being 
mindful of the headers. 
"""

HURDAT2.drop(HURDAT2.columns[8:23], axis=1, inplace=True)

In [14]:
# change to lowercase

HURDAT2['storm_name'] = HURDAT2['storm_name'].str.lower()

In [15]:
# check df shape

HURDAT2.shape

(55437, 8)

## storm identifiers level 0

In [16]:
"""
- filter for records where the storm_id = AL (atlantic basin)

raw data is multiline...
- focus here is on level0 data
- storm_id is the unqiue identifier
- storm_name is the name assigned to the storm
- associated_records are metadata records (level1)
that belong to the unique storm_id
"""

storm_level0_df = HURDAT2[HURDAT2['storm_id'].str.contains('AL*')]

In [17]:
storm_level0_df = storm_level0_df[['storm_id', 'storm_name', 'associated_records']]

In [18]:
storm_level0_df.head(2)

,storm_id,storm_name,associated_records
0,AL011851,unnamed,14
15,AL021851,unnamed,1


# flatten storm dataframe

### create a list of dfs containing metadata records per storm_id

In [19]:
# n number of records following the storm_id belong to that id
# must convert str to int

storm_num_ar = storm_level0_df['associated_records'].astype(int) 

In [20]:
"""
under each storm_id in level0 
is n number of metadata level1 records 
(associated_records).

using a loop... 
create a list of dataframes using the associated  \
record count for each storm identifier matched on \
index to the HURDAT2 data.
"""

list_dfs = []
for item in storm_num_ar.index:
    # using the index, gets the number of records \
    # associated with each storm
    associated_row_cnt = storm_num_ar.loc[item]  
    
    # using the index, selects the records associated \
    # which each storm and transforms those rows into \
    # new dfs
    new_df = pd.DataFrame(HURDAT2.loc[item+1:item + associated_row_cnt])
    
    # the headers of each new df are renamed
    new_df = new_df.rename(columns={
            'storm_id': 'storm_date',
            'storm_name': 'storm_time',
            'associated_records': 'rec_identifier',
            'storm_date': 'storm_type',
            'storm_time': 'latitude',
            'rec_identifier': 'longitude',
            'storm_type': 'max_sustained_wind(kt)',
            'latitude': 'minimum_pressure(mbar)',
    })
    
#     print(new_df)
    
    # add new columns, assign values from HURDAT2 data
    new_df['storm_id'] = HURDAT2['storm_id'][item]
    new_df['storm_name'] = HURDAT2['storm_name'][item]
    new_df['associated_records'] = HURDAT2['associated_records'][item]   

    # reorder columns
    new_df = new_df[['storm_id',
                    'storm_name',
                    'associated_records',
                    'storm_date',
                    'storm_time',
                    'rec_identifier',
                    'storm_type',
                    'latitude',
                    'longitude',
                    'max_sustained_wind(kt)',
                    'minimum_pressure(mbar)'
    ]]
    
    

    # output a list of dfs
    list_dfs.append(new_df)

In [21]:
# ensure the record count is accurate to the full set

print(len(HURDAT2))

55437


### concatentate all dfs into a single df

In [22]:
# concat the list of dfs into a single df \
# (concat since the indexes match)
# this 'flattens' the dataset

flattenStorms_df = pd.concat(list_dfs )
print(flattenStorms_df.shape)
flattenStorms_df.tail(5)

(53501, 11)


,storm_id,storm_name,associated_records,storm_date,storm_time,rec_identifier,storm_type,latitude,longitude,max_sustained_wind(kt),minimum_pressure(mbar)
55432,AL212021,wanda,54,20211107,0000,,TS,37.4N,37.4W,35.0,1003.0
55433,AL212021,wanda,54,20211107,0600,,TS,38.1N,36.4W,35.0,1004.0
55434,AL212021,wanda,54,20211107,1200,,LO,39.2N,34.9W,35.0,1006.0
55435,AL212021,wanda,54,20211107,1800,,LO,40.9N,32.8W,40.0,1006.0
55436,AL212021,wanda,54,20211108,0000,,LO,43.2N,29.7W,40.0,1006.0


### filter for: named storms

In [23]:
"""
since we're looking for a possible 
correlation to baby names, storm data 
must be filtered by named storms
"""

namedStorms_df = flattenStorms_df\
    [flattenStorms_df['storm_name']!= 'unnamed']

In [24]:
# check for named storms, no 'unnamed' storms should be seen

namedStorms_df.drop_duplicates().describe()

,max_sustained_wind(kt),minimum_pressure(mbar)
count,26634.000000,26634.000000
mean,51.744087,579.739619
std,25.432398,807.417662
min,10.000000,-999.000000
25%,30.000000,953.000000
50%,45.000000,993.000000
75%,65.000000,1005.000000
max,165.000000,1024.000000


### parse: storm_id

In [25]:
# verify length of id

def checkIdLength(df, item, anticipated_length):
    """
    check to ensure the length of the 
    id is as anticipated
    """
    for id in df[item]:
        length = len(id)
        if length != anticipated_length:
            print('Uh-oh! There is an outlier to contend with!')
        else: 
            return length

In [26]:
def decodeId(df, item, anticipated_length): 
    """
    uses checkIdLength to ensure the length of the id
    is as anticipated then parses the storm_id, creating
    a new series for basin, ATCF_cyclone_num_forYear, 
    and storm_year
    """
    if checkIdLength(df, item, anticipated_length) == anticipated_length: 
        for id in df[item]:
            df['basin'] = df[item].str[:2]
            df['ATCF_cyclone_num_forYear'] = df[item].str[2:4]
            df['storm_year'] = df[item].str[4:8].astype('int64')
            return df
    else: 
        print('Uh-oh! There is an outlier to contend with!')

In [27]:
namedStorms_df = namedStorms_df.copy()
decodeId(namedStorms_df, 'storm_id', 8)

,storm_id,storm_name,associated_records,storm_date,storm_time,rec_identifier,storm_type,latitude,longitude,max_sustained_wind(kt),minimum_pressure(mbar),basin,ATCF_cyclone_num_forYear,storm_year
22686,AL011950,able,51,19500812,0000,,TS,17.1N,55.5W,35.0,-999.0,AL,01,1950
22687,AL011950,able,51,19500812,0600,,TS,17.7N,56.3W,40.0,-999.0,AL,01,1950
22688,AL011950,able,51,19500812,1200,,TS,18.2N,57.4W,45.0,-999.0,AL,01,1950
22689,AL011950,able,51,19500812,1800,,TS,19.0N,58.6W,50.0,-999.0,AL,01,1950
22690,AL011950,able,51,19500813,0000,,TS,20.0N,60.0W,50.0,-999.0,AL,01,1950
22691,AL011950,able,51,19500813,0600,,TS,20.7N,61.1W,50.0,-999.0,AL,01,1950
22692,AL011950,able,51,19500813,1200,,TS,21.3N,62.2W,55.0,-999.0,AL,01,1950
22693,AL011950,able,51,19500813,1800,,TS,22.0N,63.2W,55.0,997.0,AL,01,1950
22694,AL011950,able,51,19500814,0000,,TS,22.7N,63.8W,60.0,995.0,AL,01,1950
22695,AL011950,able,51,19500814,0600,,TS,23.1N,64.6W,60.0,-999.0,AL,01,1950


### cleanup: time

In [28]:
def convertTime(df, item):    
    """
    transforms time to datetime
    in a series
    """
    df[item] = df[item].apply(lambda x:pd\
                       .to_datetime(str(x), format='%H%M'))\
                       .dt.time
    return df

In [29]:
namedStorms_df = namedStorms_df.copy()
convertTime(namedStorms_df, 'storm_time')

,storm_id,storm_name,associated_records,storm_date,storm_time,rec_identifier,storm_type,latitude,longitude,max_sustained_wind(kt),minimum_pressure(mbar),basin,ATCF_cyclone_num_forYear,storm_year
22686,AL011950,able,51,19500812,00:00:00,,TS,17.1N,55.5W,35.0,-999.0,AL,01,1950
22687,AL011950,able,51,19500812,06:00:00,,TS,17.7N,56.3W,40.0,-999.0,AL,01,1950
22688,AL011950,able,51,19500812,12:00:00,,TS,18.2N,57.4W,45.0,-999.0,AL,01,1950
22689,AL011950,able,51,19500812,18:00:00,,TS,19.0N,58.6W,50.0,-999.0,AL,01,1950
22690,AL011950,able,51,19500813,00:00:00,,TS,20.0N,60.0W,50.0,-999.0,AL,01,1950
22691,AL011950,able,51,19500813,06:00:00,,TS,20.7N,61.1W,50.0,-999.0,AL,01,1950
22692,AL011950,able,51,19500813,12:00:00,,TS,21.3N,62.2W,55.0,-999.0,AL,01,1950
22693,AL011950,able,51,19500813,18:00:00,,TS,22.0N,63.2W,55.0,997.0,AL,01,1950
22694,AL011950,able,51,19500814,00:00:00,,TS,22.7N,63.8W,60.0,995.0,AL,01,1950
22695,AL011950,able,51,19500814,06:00:00,,TS,23.1N,64.6W,60.0,-999.0,AL,01,1950


### parse: date

In [30]:
def parseDate(df, date):
    """
    parses the storm_date into a series 
    for data (day, month and year, if needed)
    """
    df[date] = df[date].apply(lambda x:pd.to_datetime(str(x)))
#     df[day] = df[date].dt.day
#     df[month] = df[date].dt.month
#     df[year] = df[date].dt.year
    return df

In [31]:
namedStorms_df = namedStorms_df.copy()
parseDate(namedStorms_df, 
          'storm_date')

,storm_id,storm_name,associated_records,storm_date,storm_time,rec_identifier,storm_type,latitude,longitude,max_sustained_wind(kt),minimum_pressure(mbar),basin,ATCF_cyclone_num_forYear,storm_year
22686,AL011950,able,51,1950-08-12,00:00:00,,TS,17.1N,55.5W,35.0,-999.0,AL,01,1950
22687,AL011950,able,51,1950-08-12,06:00:00,,TS,17.7N,56.3W,40.0,-999.0,AL,01,1950
22688,AL011950,able,51,1950-08-12,12:00:00,,TS,18.2N,57.4W,45.0,-999.0,AL,01,1950
22689,AL011950,able,51,1950-08-12,18:00:00,,TS,19.0N,58.6W,50.0,-999.0,AL,01,1950
22690,AL011950,able,51,1950-08-13,00:00:00,,TS,20.0N,60.0W,50.0,-999.0,AL,01,1950
22691,AL011950,able,51,1950-08-13,06:00:00,,TS,20.7N,61.1W,50.0,-999.0,AL,01,1950
22692,AL011950,able,51,1950-08-13,12:00:00,,TS,21.3N,62.2W,55.0,-999.0,AL,01,1950
22693,AL011950,able,51,1950-08-13,18:00:00,,TS,22.0N,63.2W,55.0,997.0,AL,01,1950
22694,AL011950,able,51,1950-08-14,00:00:00,,TS,22.7N,63.8W,60.0,995.0,AL,01,1950
22695,AL011950,able,51,1950-08-14,06:00:00,,TS,23.1N,64.6W,60.0,-999.0,AL,01,1950


### decode: storm_type

In [32]:
def decodeFromDict(abrvs, decode, df, item):    
    """
    using a list of abbrevations and a list
    of decodes, zips together the two lists
    into an ordered dict, then maps the 
    decoded values to a new series
    """
    from collections import OrderedDict as od
    
    # returns numpy object converted to a list
    x = od(zip(abrvs,decode))

    for abrv in abrvs: 
        df[item] = df[item].map(x)
        return df

In [33]:
storm_type_abrv = namedStorms_df["storm_type"].drop_duplicates().tolist()
print(storm_type_abrv)

decode = ['tropical storm',
          'hurricane',          
          'extratropical storm', 
          'tropical depression', 
          'disturbance(any)',
          'subtropical depression', 
          'subtropical storm', 
          'low - not tropical-subtropical-or-extratropical',
          'tropical wave']

decodeFromDict(storm_type_abrv, decode, namedStorms_df, 'storm_type')

['TS', 'HU', 'EX', 'TD', 'DB', 'SD', 'SS', 'LO', 'WV']


,storm_id,storm_name,associated_records,storm_date,storm_time,rec_identifier,storm_type,latitude,longitude,max_sustained_wind(kt),minimum_pressure(mbar),basin,ATCF_cyclone_num_forYear,storm_year
22686,AL011950,able,51,1950-08-12,00:00:00,,tropical storm,17.1N,55.5W,35.0,-999.0,AL,01,1950
22687,AL011950,able,51,1950-08-12,06:00:00,,tropical storm,17.7N,56.3W,40.0,-999.0,AL,01,1950
22688,AL011950,able,51,1950-08-12,12:00:00,,tropical storm,18.2N,57.4W,45.0,-999.0,AL,01,1950
22689,AL011950,able,51,1950-08-12,18:00:00,,tropical storm,19.0N,58.6W,50.0,-999.0,AL,01,1950
22690,AL011950,able,51,1950-08-13,00:00:00,,tropical storm,20.0N,60.0W,50.0,-999.0,AL,01,1950
22691,AL011950,able,51,1950-08-13,06:00:00,,tropical storm,20.7N,61.1W,50.0,-999.0,AL,01,1950
22692,AL011950,able,51,1950-08-13,12:00:00,,tropical storm,21.3N,62.2W,55.0,-999.0,AL,01,1950
22693,AL011950,able,51,1950-08-13,18:00:00,,tropical storm,22.0N,63.2W,55.0,997.0,AL,01,1950
22694,AL011950,able,51,1950-08-14,00:00:00,,tropical storm,22.7N,63.8W,60.0,995.0,AL,01,1950
22695,AL011950,able,51,1950-08-14,06:00:00,,tropical storm,23.1N,64.6W,60.0,-999.0,AL,01,1950


In [34]:
# test 

# from collections import OrderedDict as od
# storm_type_abrv = namedStorms_df["storm_type"].drop_duplicates().tolist()

# decode = ['tropical storm',
#           'hurricane',
#           'extratropical storm', 
#           'tropical depression', 
#           'low - not tropical-subtropical-or-extratropical',
#           'disturbance(any)',
#           'subtropical depression', 
#           'subtropical storm',  
#           'tropical wave']

# qc = od(zip(storm_type_abrv,decode))
# qc

In [35]:
# Emily is a use case that has a hurricane record

namedStorms_df[(namedStorms_df['storm_name']=='emily') \
               & (namedStorms_df['storm_type']=='hurricane') \
               \
               # storm_year as str
#                & (namedStorms_df['storm_year']== '1987') \
               \
               # storm_year as int (should be int)
               & (namedStorms_df['storm_year']== 1987) \
               & (namedStorms_df['max_sustained_wind(kt)'] > 100)\
              ]

,storm_id,storm_name,associated_records,storm_date,storm_time,rec_identifier,storm_type,latitude,longitude,max_sustained_wind(kt),minimum_pressure(mbar),basin,ATCF_cyclone_num_forYear,storm_year
38220,AL121987,emily,30,1987-09-22,18:00:00,,hurricane,16.7N,69.1W,110.0,958.0,AL,12,1987
38221,AL121987,emily,30,1987-09-23,00:00:00,,hurricane,17.8N,70.4W,105.0,960.0,AL,12,1987


### decode: rec_identifier

In [36]:
namedStorms_df['rec_identifier'].unique()

array(['', 'L', 'R', 'I', 'P', 'T', 'W', 'C', 'S', 'G'], dtype=object)

In [37]:
# decode the rec_identifier values

storm_rec_identifier_abrv = namedStorms_df["rec_identifier"].drop_duplicates().tolist()

decode = ['',
          'landfall',
          'additional detail on the intensity when rapid changes are underway', 
          'intensity peak', 
          'minimum in central pressure',
          'additional detail on the position',
          'maximum sustained wind speed ', 
          'closest approach to a coast',  
          'status change in system',
          'genesis'
         ]

decodeFromDict(storm_rec_identifier_abrv, decode, namedStorms_df, 'rec_identifier')

,storm_id,storm_name,associated_records,storm_date,storm_time,rec_identifier,storm_type,latitude,longitude,max_sustained_wind(kt),minimum_pressure(mbar),basin,ATCF_cyclone_num_forYear,storm_year
22686,AL011950,able,51,1950-08-12,00:00:00,,tropical storm,17.1N,55.5W,35.0,-999.0,AL,01,1950
22687,AL011950,able,51,1950-08-12,06:00:00,,tropical storm,17.7N,56.3W,40.0,-999.0,AL,01,1950
22688,AL011950,able,51,1950-08-12,12:00:00,,tropical storm,18.2N,57.4W,45.0,-999.0,AL,01,1950
22689,AL011950,able,51,1950-08-12,18:00:00,,tropical storm,19.0N,58.6W,50.0,-999.0,AL,01,1950
22690,AL011950,able,51,1950-08-13,00:00:00,,tropical storm,20.0N,60.0W,50.0,-999.0,AL,01,1950
22691,AL011950,able,51,1950-08-13,06:00:00,,tropical storm,20.7N,61.1W,50.0,-999.0,AL,01,1950
22692,AL011950,able,51,1950-08-13,12:00:00,,tropical storm,21.3N,62.2W,55.0,-999.0,AL,01,1950
22693,AL011950,able,51,1950-08-13,18:00:00,,tropical storm,22.0N,63.2W,55.0,997.0,AL,01,1950
22694,AL011950,able,51,1950-08-14,00:00:00,,tropical storm,22.7N,63.8W,60.0,995.0,AL,01,1950
22695,AL011950,able,51,1950-08-14,06:00:00,,tropical storm,23.1N,64.6W,60.0,-999.0,AL,01,1950


## reference: saffir-simpson wind scale
https://www.nhc.noaa.gov/aboutsshws.php#:~:text=The%20Saffir%2DSimpson%20Hurricane%20Wind,Scale%20estimates%20potential%20property%20damage.

In [38]:
"""
NURDAT2 data contains no reference to
category. In order to identify category, 
the Saffir_Simpson Wind Scale must be 
referenced. The max_sustained_wind(kt)
can be measured against the min-max of 
each category's range to set the 
category, where the storm_type is 
hurricane

"""

data = [{'category': 1, 
         'sustained_wind(kt)': '64-82', 
         'max_sustained_wind(kt)': 82, 
         'min_sustained_wind(kt)': 64,
         'sustained_wind(mph)': '74-95', 
         'brief_damage_description': \
         'Power outages that could last a few to several days.'},
        
       {'category': 2, 
        'sustained_wind(kt)': '83-95', 
        'max_sustained_wind(kt)': 95, 
        'min_sustained_wind(kt)': 83,
        'sustained_wind(mph)': '96-110', 
        'brief_damage_description': \
        'Near-total power loss is expected \
        with outages that could last from several days to weeks.'},
        
       {'category': 3, 
        'sustained_wind(kt)': '96-112', 
        'max_sustained_wind(kt)': 112, 
        'min_sustained_wind(kt)': 96,
        'sustained_wind(mph)': '111-129', 
        'brief_damage_description': \
        'Electricity and water will be \
        unavailable for several days to weeks after the storm passes.'},
        
       {'category': 4,
        'sustained_wind(kt)': '113-136', 
        'max_sustained_wind(kt)': 136, 
        'min_sustained_wind(kt)': 113,
        'sustained_wind(mph)': '130-156', 
        'brief_damage_description': \
        'Catastrophic damage will occur; most of \
        the area will be uninhabitable for weeks or months.'},
        
       {'category': 5,
        'sustained_wind(kt)': '137+', 
        'min_sustained_wind(kt)': 137, 
        'sustained_wind(mph)': '157+',
        'brief_damage_description': \
        'Catastrophic damage will occur; most of the \
        area will be uninhabitable for weeks or months.'}]


saffir_simpson_scale = pd.DataFrame(data, 
                                    columns=['category', 
                                             'min_sustained_wind(kt)',
                                             'max_sustained_wind(kt)',  
                                             'sustained_wind(kt)',
                                             'brief_damage_description',]) \
                                             .fillna(0)

saffir_simpson_scale['max_sustained_wind(kt)'] = saffir_simpson_scale['max_sustained_wind(kt)'].astype(int)
saffir_simpson_scale['max_sustained_wind(kt)']= np.round(saffir_simpson_scale['max_sustained_wind(kt)'], decimals=0)
saffir_simpson_scale

,category,min_sustained_wind(kt),max_sustained_wind(kt),sustained_wind(kt),brief_damage_description
0,1,64,82,64-82,Power outages that could last a few to several...
1,2,83,95,83-95,Near-total power loss is expected with...
2,3,96,112,96-112,Electricity and water will be unavaila...
3,4,113,136,113-136,Catastrophic damage will occur; most of ...
4,5,137,0,137+,Catastrophic damage will occur; most of the ...


### category assignment

In [39]:
# # https://simple.wikipedia.org/wiki/Saffir%E2%80%93Simpson_scale

# def categoryAssignment_HU(df, category_item, storm_type_item, knots_item ):
#     df[category_item] = 0

#     df.reset_index()
#     for index, row in df.iterrows():
#         if row[storm_type_item] == 'hurricane':
#             if row[knots_item] <= 82:
#                 df.loc[[index],[category_item]] = 1
#             elif  (row[knots_item] >= 83) & (row[knots_item] <= 95):
#                 df.loc[[index],[category_item]] = 2
#             elif  (row[knots_item] >= 96) & (row[knots_item] <= 112):
#                 df.loc[[index],[category_item]] = 3
#             elif  (row[knots_item] >= 113) & (row[knots_item] <= 136):
#                 df.loc[[index],[category_item]] = 4
#             elif row[knots_item] >= 137:
#                 df.loc[[index],[category_item]] = 5                

In [40]:
def categoryAssignment_HU(df, storm_type_item, max_kts_item):
    """
    determines whether the max_sustained_wind(kt) 
    fall into a saffir-simpson wind scale 
    category and assigns the category if the
    record is a hurricane.
    
    """
    
    conditions = [
    (
        (df[storm_type_item] != 'hurricane')
    ),
    (
        (df[storm_type_item] == 'hurricane') &
        (df[max_kts_item] < saffir_simpson_scale['min_sustained_wind(kt)'][0])
    ),
    (
        (df[storm_type_item] == 'hurricane') &
        (df[max_kts_item] <= saffir_simpson_scale['max_sustained_wind(kt)'][0]) &
        (df[max_kts_item] > saffir_simpson_scale['min_sustained_wind(kt)'][0])
    ),
    (
        (df[storm_type_item] == 'hurricane') &
        ((df[max_kts_item] <= saffir_simpson_scale['max_sustained_wind(kt)'][1]) & 
        (df[max_kts_item] > saffir_simpson_scale['min_sustained_wind(kt)'][1]))
    ),

    (
        (df[storm_type_item] == 'hurricane') &
        ((df[max_kts_item] <= saffir_simpson_scale['max_sustained_wind(kt)'][2]) &
        (df[max_kts_item] > saffir_simpson_scale['min_sustained_wind(kt)'][2]))
    ),

    (
        (df[storm_type_item] == 'hurricane') &
        ((df[max_kts_item] <= saffir_simpson_scale['max_sustained_wind(kt)'][3]) &
        (df[max_kts_item] > saffir_simpson_scale['min_sustained_wind(kt)'][3]))
    ),
    (
        (df[storm_type_item] == 'hurricane') &
        (df[max_kts_item] >= saffir_simpson_scale['min_sustained_wind(kt)'][4])
    )
    ]

    choices = [df[storm_type_item],
               'not categorized',
              saffir_simpson_scale['category'][0],
              saffir_simpson_scale['category'][1],
              saffir_simpson_scale['category'][2],
              saffir_simpson_scale['category'][3],
              saffir_simpson_scale['category'][4]]

    df['category'] = np.select(conditions, choices)
    return df

In [41]:
namedStorms_df = namedStorms_df.copy()
categoryAssignment_HU(namedStorms_df, 'storm_type', 'max_sustained_wind(kt)')

,storm_id,storm_name,associated_records,storm_date,storm_time,rec_identifier,storm_type,latitude,longitude,max_sustained_wind(kt),minimum_pressure(mbar),basin,ATCF_cyclone_num_forYear,storm_year,category
22686,AL011950,able,51,1950-08-12,00:00:00,,tropical storm,17.1N,55.5W,35.0,-999.0,AL,01,1950,tropical storm
22687,AL011950,able,51,1950-08-12,06:00:00,,tropical storm,17.7N,56.3W,40.0,-999.0,AL,01,1950,tropical storm
22688,AL011950,able,51,1950-08-12,12:00:00,,tropical storm,18.2N,57.4W,45.0,-999.0,AL,01,1950,tropical storm
22689,AL011950,able,51,1950-08-12,18:00:00,,tropical storm,19.0N,58.6W,50.0,-999.0,AL,01,1950,tropical storm
22690,AL011950,able,51,1950-08-13,00:00:00,,tropical storm,20.0N,60.0W,50.0,-999.0,AL,01,1950,tropical storm
22691,AL011950,able,51,1950-08-13,06:00:00,,tropical storm,20.7N,61.1W,50.0,-999.0,AL,01,1950,tropical storm
22692,AL011950,able,51,1950-08-13,12:00:00,,tropical storm,21.3N,62.2W,55.0,-999.0,AL,01,1950,tropical storm
22693,AL011950,able,51,1950-08-13,18:00:00,,tropical storm,22.0N,63.2W,55.0,997.0,AL,01,1950,tropical storm
22694,AL011950,able,51,1950-08-14,00:00:00,,tropical storm,22.7N,63.8W,60.0,995.0,AL,01,1950,tropical storm
22695,AL011950,able,51,1950-08-14,06:00:00,,tropical storm,23.1N,64.6W,60.0,-999.0,AL,01,1950,tropical storm


In [42]:
namedStorms_df['category'].drop_duplicates()

22686                                     tropical storm
22697                                                  1
22705                                                  2
22708                                                  3
22727                                extratropical storm
22738                                tropical depression
22865                                                  4
24115                                                  5
25992                                   disturbance(any)
26116                                    not categorized
28851                             subtropical depression
28854                                  subtropical storm
29315    low - not tropical-subtropical-or-extratropical
36124                                      tropical wave
Name: category, dtype: object

In [43]:
# verify category assignment is working as anticipated...

namedStorms_df[ \
        (namedStorms_df['storm_type'] == 'hurricane') &  \
        (namedStorms_df['max_sustained_wind(kt)'] == 110) & \
        (namedStorms_df['storm_name'] == 'emily')
              ]

,storm_id,storm_name,associated_records,storm_date,storm_time,rec_identifier,storm_type,latitude,longitude,max_sustained_wind(kt),minimum_pressure(mbar),basin,ATCF_cyclone_num_forYear,storm_year,category
38220,AL121987,emily,30,1987-09-22,18:00:00,,hurricane,16.7N,69.1W,110.0,958.0,AL,12,1987,3
46110,AL052005,emily,45,2005-07-15,00:00:00,,hurricane,13.3N,66.7W,110.0,959.0,AL,05,2005,3
46114,AL052005,emily,45,2005-07-16,00:00:00,,hurricane,14.9N,73.4W,110.0,958.0,AL,05,2005,3
46131,AL052005,emily,45,2005-07-20,00:00:00,,hurricane,24.4N,96.1W,110.0,948.0,AL,05,2005,3
46132,AL052005,emily,45,2005-07-20,06:00:00,,hurricane,24.6N,96.9W,110.0,944.0,AL,05,2005,3
46133,AL052005,emily,45,2005-07-20,12:00:00,landfall,hurricane,24.8N,97.6W,110.0,944.0,AL,05,2005,3


### cleanup: coordinates

In [44]:
def transformCoordinates(df, lat_item, long_item ):
    """
    removes char from lat and long 
    converts lat and long to int
    converts S and W to negative int
    zips transformed lat and long into
    a list of coordinate tuples
    """
    lat_n = pd.to_numeric((df[lat_item].str[:-1]), errors='coerce')
    lat_s = pd.to_numeric((df[lat_item].str[:-1]), errors='coerce')*-1
    long_e = pd.to_numeric((df[long_item].str[:-1]), errors='coerce')
    long_w = pd.to_numeric((df[long_item].str[:-1]), errors='coerce')*-1

    df[lat_item] = np.where(len(df[lat_item].str[-1] =='N'), lat_n, lat_s)  
    df[long_item] = np.where(len(df[long_item].str[-1] =='W'), long_w, long_e)
    df['coordinates'] = list(zip(df[lat_item], df[long_item]))
    return df

In [45]:
transformCoordinates(namedStorms_df, 'latitude', 'longitude')

,storm_id,storm_name,associated_records,storm_date,storm_time,rec_identifier,storm_type,latitude,longitude,max_sustained_wind(kt),minimum_pressure(mbar),basin,ATCF_cyclone_num_forYear,storm_year,category,coordinates
22686,AL011950,able,51,1950-08-12,00:00:00,,tropical storm,17.1,-55.5,35.0,-999.0,AL,01,1950,tropical storm,"(17.1, -55.5)"
22687,AL011950,able,51,1950-08-12,06:00:00,,tropical storm,17.7,-56.3,40.0,-999.0,AL,01,1950,tropical storm,"(17.7, -56.3)"
22688,AL011950,able,51,1950-08-12,12:00:00,,tropical storm,18.2,-57.4,45.0,-999.0,AL,01,1950,tropical storm,"(18.2, -57.4)"
22689,AL011950,able,51,1950-08-12,18:00:00,,tropical storm,19.0,-58.6,50.0,-999.0,AL,01,1950,tropical storm,"(19.0, -58.6)"
22690,AL011950,able,51,1950-08-13,00:00:00,,tropical storm,20.0,-60.0,50.0,-999.0,AL,01,1950,tropical storm,"(20.0, -60.0)"
22691,AL011950,able,51,1950-08-13,06:00:00,,tropical storm,20.7,-61.1,50.0,-999.0,AL,01,1950,tropical storm,"(20.7, -61.1)"
22692,AL011950,able,51,1950-08-13,12:00:00,,tropical storm,21.3,-62.2,55.0,-999.0,AL,01,1950,tropical storm,"(21.3, -62.2)"
22693,AL011950,able,51,1950-08-13,18:00:00,,tropical storm,22.0,-63.2,55.0,997.0,AL,01,1950,tropical storm,"(22.0, -63.2)"
22694,AL011950,able,51,1950-08-14,00:00:00,,tropical storm,22.7,-63.8,60.0,995.0,AL,01,1950,tropical storm,"(22.7, -63.8)"
22695,AL011950,able,51,1950-08-14,06:00:00,,tropical storm,23.1,-64.6,60.0,-999.0,AL,01,1950,tropical storm,"(23.1, -64.6)"


# 3. State reference table

51 records...  

json file that contains the state name, 
state abbrevation and the state code for all
50 US statues  

To be used as a reference table to map coordinates

In [46]:
# https://worldpopulationreview.com/states/state-abbreviations

"""
reads in json of state abbrevations for reference
drops unneeded columns
converts to a df
"""

with open('/home/workspace/source_states/states.json') as json_file:
    states_json_data = json.load(json_file)
    for item in states_json_data:
        del item['Abbrev']

state_ref = pd.DataFrame(states_json_data).rename(str.lower, axis='columns').reset_index(drop = True)

state_ref['state_id'] = state_ref.index

state_ref = state_ref[['state_id',
                      'code',
                      'state']]

state_ref.head(5)

,state_id,code,state
0,0,AL,Alabama
1,1,AK,Alaska
2,2,AZ,Arizona
3,3,AR,Arkansas
4,4,CA,California


## map coordinates to state

In [47]:
"""
this transformation was made possible 
through collaboration with:
Jasmine Cawley and Jason Petruno.

relies on geopy and an open source map API, to get
the state code from an address by doing a reverse lookup 
iterating through tuples of storm coordinates where
the storm made landfall, then maps that state code
to the list of 'states we care about' from a dict of
states made from the state reference table, then inserts
a new series where the records match on index. 

expect 5-10 minutes to run. 
"""

! pip install pygeocoder
! pip install geopy

def geolocCoordinates_toState(df):
    import certifi
    import ssl
    from geopy.geocoders import Nominatim
    import geopy

    ctx = ssl.create_default_context(cafile=certifi.where())
    geopy.geocoders.options.default_ssl_context = ctx
    geolocator = Nominatim(user_agent = 'http', timeout=None)

    state_withCode = dict(zip(state_ref['state'], state_ref['code']))
    states_we_care_about = state_withCode.keys()

    storm_state_code = None
    no_state = 0
    data = []

    # namedStorms_df[storm_state] = ""
    df['storm_state_code'] = ""

    df.reset_index()
    for index, row in df.iterrows():

        # If the storm datapoint indicates landfall
        if row['rec_identifier'] == 'landfall':
#         if row['storm_type'] == 'hurricane':

            #Get a location object fort this row's lat//lng 
            location = geolocator.reverse(row['coordinates'])

            # defensive coding! 
            if location is not None:
                storm_state = location.raw.get('address').get('state')
    #             print(f"{storm_state}")

                # make sure we're in the 50 states so we can get the state's code, otherwise keep going
                if storm_state not in states_we_care_about:
                    no_state +=1
                    continue
                else:
                    storm_state_code = state_withCode[storm_state]

                    # now we can add the state and the code back into the data frame
        #             namedStorms_df.loc[[index],[storm_state]] = storm_state
                    df.loc[[index],['storm_state_code']] = storm_state_code
        else:
            no_state +=1
    #         print('Not applicable to the US')

  Running setup.py bdist_wheel for pygeocoder ... done
  Stored in directory: /root/.cache/pip/wheels/7c/4c/00/d05c66c4af5411c554c91b8079732c8a0359c2226fb8c01031
Successfully built pygeocoder
    100% |████████████████████████████████| 122kB 4.8MB/s ta 0:00:01


In [48]:
# # takes a bit but it works...
# # namedStorms_df full dataset I've been testing with...

geolocCoordinates_toState(namedStorms_df)

/opt/conda/lib/python3.6/site-packages/geopy/adapters.py:639: UserWarning: urllib3 prior to 1.24.2 is known to have a bug with custom ssl contexts: it attempts to load system certificates to them. Please consider upgrading `requests` and `urllib3` packages. See https://github.com/urllib3/urllib3/pull/1566
  UserWarning,


# Build Schema Tables

![Schema](md_images/schemaCapstone.png) 

## saffir_simpson_hurricane_wind_scale_ref

In [49]:
saffir_simpson_scale

,category,min_sustained_wind(kt),max_sustained_wind(kt),sustained_wind(kt),brief_damage_description
0,1,64,82,64-82,Power outages that could last a few to several...
1,2,83,95,83-95,Near-total power loss is expected with...
2,3,96,112,96-112,Electricity and water will be unavaila...
3,4,113,136,113-136,Catastrophic damage will occur; most of ...
4,5,137,0,137+,Catastrophic damage will occur; most of the ...


## state_ref

In [50]:
state_ref.head(5)

,state_id,code,state
0,0,AL,Alabama
1,1,AK,Alaska
2,2,AZ,Arizona
3,3,AR,Arkansas
4,4,CA,California


In [51]:
state_ref[['state']].drop_duplicates().describe()

,state
count,51
unique,51
top,Oklahoma
freq,1


## babyNames_byState_fact

In [52]:
namesbystate_df.tail(5)

,babyName_id,birth_year,birth_name,popularity,birth_state_code,gender
6311499,6311499,2021,wilson,5,SC,M
6311500,6311500,2021,wren,5,SC,M
6311501,6311501,2021,xzavier,5,SC,M
6311502,6311502,2021,zakari,5,SC,M
6311503,6311503,2021,zymir,5,SC,M


In [53]:
namesbystate_df[['birth_state_code', 'birth_name']].drop_duplicates().describe()

,birth_state_code,birth_name
count,310308,310308
unique,51,32403
top,CA,jaclyn
freq,20239,51


## storms_byName_dim

In [54]:
storms_byName_dim = namedStorms_df[['storm_id',
                               'basin',
                               'ATCF_cyclone_num_forYear',
                               'storm_year',
                               'storm_name',
                               'associated_records'                             
                              ]].drop_duplicates().reset_index(drop = True)

storms_byName_dim.head()

,storm_id,basin,ATCF_cyclone_num_forYear,storm_year,storm_name,associated_records
0,AL011950,AL,01,1950,able,51
1,AL021950,AL,02,1950,baker,60
2,AL031950,AL,03,1950,charlie,61
3,AL041950,AL,04,1950,dog,74
4,AL051950,AL,05,1950,easy,39


In [55]:
storms_byName_dim[['storm_id']].drop_duplicates().describe()

,storm_id
count,821
unique,821
top,AL122005
freq,1


## storms_location_dim

In [56]:
storms_location_dim = namedStorms_df[[
               'storm_id',
               'storm_name',
               'storm_year',
               'storm_type',
               'storm_state_code'
              ]].drop_duplicates() \
                .reset_index(drop = True)

storms_location_dim = storms_location_dim[ \
    (storms_location_dim['storm_state_code'] !="") 
]
storms_location_dim = storms_location_dim.groupby(["storm_id", 
                                   "storm_name",
                                   "storm_type",
                                   "storm_state_code"])\
                            ['storm_year'].max()\
                            .to_frame() \
                            .reset_index() \
                            .drop_duplicates()

storms_location_dim['location_id'] = storms_location_dim.index

storms_location_dim = storms_location_dim[['location_id',              
                    'storm_id',
                    'storm_name',
                    'storm_year',
                    'storm_type',
                    'storm_state_code']]

storms_location_dim.head()

,location_id,storm_id,storm_name,storm_year,storm_type,storm_state_code
0,0,AL011953,alice,1953,tropical storm,FL
1,1,AL011955,brenda,1955,tropical storm,LA
2,2,AL011959,arlene,1959,tropical storm,LA
3,3,AL011966,alma,1966,hurricane,FL
4,4,AL011968,abby,1968,tropical storm,FL


In [57]:
storms_location_dim[['storm_name', 'storm_id']].drop_duplicates().describe()

,storm_name,storm_id
count,200,200
unique,141,200
top,bonnie,AL122005
freq,5,1


In [58]:
# # test - use: bonnie (5), andrew (2), sandy(1), elsa(3) and/or katrina(2)

storms_location_dim[storms_location_dim['storm_name']=='emily']

,location_id,storm_id,storm_name,storm_year,storm_type,storm_state_code
123,123,AL062017,emily,2017,tropical storm,FL


## storms_time_dim

In [59]:
# storms_time_dim = namedStorms_df[[
#         'storm_id',
#         'storm_name',
#         'storm_type',
#         'storm_date',        
#         'storm_year',
#         'storm_month',
#         'storm_day',
#         'storm_time'
#         ]].drop_duplicates()

# storms_time_dim = storms_time_dim \
#     .groupby([ \
#             "storm_id", 
#             "storm_name",
#             "storm_type",
#             'storm_date',        
#             'storm_year',
#             'storm_month',
#             'storm_day'              
#              ]) \
#             ['storm_time'].max() \
#     .to_frame() \
#     .reset_index() \
#     .drop_duplicates()

# storms_time_dim['time_id'] = storms_time_dim.index

# storms_time_dim = storms_time_dim[[ \
#             'time_id',              
#             "storm_id", 
#             "storm_name",
#             "storm_type",
#             'storm_date',        
#             'storm_year',
#             'storm_month',
#             'storm_day',
#             'storm_time']]

# storms_time_dim.head()

In [60]:
# storms_time_dim[['storm_name', 'storm_id']].drop_duplicates().describe()
# storms_time_dim[storms_time_dim['storm_name']=='arlene']

## storm_severity_dim

In [61]:
storms_severity_dim = namedStorms_df[[
            'storm_id', 
            'storm_name',
            'storm_year',
#             'rec_identifier',
#             'storm_type',
            'category',
            'max_sustained_wind(kt)',
            'minimum_pressure(mbar)'     
        ]].drop_duplicates()


# storms_severity_dim = storms_severity_dim[    
# #     ( (storms_severity_dim['rec_identifier'] == 'landfall') |
# #       (storms_severity_dim['rec_identifier'] == '') 
# #     ) &
#     ( (storms_severity_dim['category']== 0 ) |
#       (storms_severity_dim['category']== 1 ) |
#       (storms_severity_dim['category']== 2 ) | 
#       (storms_severity_dim['category']== 3 ) |
#       (storms_severity_dim['category']== 4 ) |
#       (storms_severity_dim['category']== 5 ) 
#     ) 
# ]

storms_severity_dim = storms_severity_dim \
    .groupby(['storm_id', 
             'storm_name',
             'storm_year',
#              'storm_type',
#              'rec_identifier',
              'category',
#              'max_sustained_wind(kt)',
#              'minimum_pressure(mbar)'     
              ]) \
              ['max_sustained_wind(kt)'].max() \
    .to_frame() \
    .reset_index() \
    .drop_duplicates()

storms_severity_dim['severity_id'] = storms_severity_dim.index

storms_severity_dim = storms_severity_dim[[ \
            'severity_id',              
            'storm_id', 
            'storm_name',
            'storm_year',
#             'storm_type',
#             'rec_identifier',
            'category',
            'max_sustained_wind(kt)',
#             'minimum_pressure(mbar)'
        ]]
    
storms_severity_dim.head()

,severity_id,storm_id,storm_name,storm_year,category,max_sustained_wind(kt)
0,0,AL011950,able,1950,1,80.0
1,1,AL011950,able,1950,2,95.0
2,2,AL011950,able,1950,3,110.0
3,3,AL011950,able,1950,extratropical storm,35.0
4,4,AL011950,able,1950,tropical storm,60.0


In [62]:
storms_severity_dim[['storm_name', 'storm_id', 'category']].drop_duplicates().describe()

,storm_name,storm_id,category
count,3322,3322,3322
unique,311,821,14
top,florence,AL132019,tropical storm
freq,40,9,788


In [63]:
storms_severity_dim[storms_severity_dim['storm_name']=='emily']

,severity_id,storm_id,storm_name,storm_year,category,max_sustained_wind(kt)
841,841,AL051993,emily,1993,1,80.0
842,842,AL051993,emily,1993,2,95.0
843,843,AL051993,emily,1993,3,100.0
844,844,AL051993,emily,1993,extratropical storm,25.0
845,845,AL051993,emily,1993,tropical depression,30.0
846,846,AL051993,emily,1993,tropical storm,60.0
887,887,AL052005,emily,2005,1,80.0
888,888,AL052005,emily,2005,2,95.0
889,889,AL052005,emily,2005,3,110.0
890,890,AL052005,emily,2005,4,135.0


## storms_metadata_fact

In [64]:
s_nm = storms_byName_dim[['storm_id', 
                          'storm_name',
                          'storm_year']]

sev = storms_severity_dim[['storm_id',
                           'storm_name',
                           'storm_year',
#                            'storm_type',
                           'severity_id',
                           'category'
                          ]]

loc = storms_location_dim[['storm_id'
                           ,'storm_name'
                           ,'storm_year'
                           ,'location_id'
                           ,'storm_state_code'
                          ]]

# remove time as dim table
# time = storms_time_dim[['storm_id'
#                        ,'storm_name'
#                        ,'storm_year'
#                        ,'time_id'
#                           ]]

storms_meta_fact = s_nm.merge(sev,on=['storm_id', 
                                      'storm_name', 
#                                       'storm_year'
                                     ])\
    .merge(loc,on=['storm_id', 
                   'storm_name',
#                    'storm_year'
                  ]) \
    .reset_index() \
    .drop_duplicates()


storms_meta_fact = storms_meta_fact \
    .groupby(['storm_id', 
             'storm_name',
             'storm_year',
#              'storm_type',
             'severity_id',
             'location_id',
             'storm_state_code'
              ]) \
              ['category'].max() \
    .to_frame() \
    .reset_index() \
    .drop_duplicates()

storms_meta_fact['storm_meta_id'] = storms_meta_fact.index

storms_meta_fact = storms_meta_fact[[  
            'storm_meta_id',  
            'storm_id', 
            'storm_name',
            'storm_year',
#             'storm_type',
            'severity_id',
            'category',
            'location_id',
            'storm_state_code' \
            ]]
    

storms_meta_fact.head()

,storm_meta_id,storm_id,storm_name,storm_year,severity_id,category,location_id,storm_state_code
0,0,AL011953,alice,1953,5,tropical depression,0,FL
1,1,AL011953,alice,1953,6,tropical storm,0,FL
2,2,AL011955,brenda,1955,7,tropical depression,1,LA
3,3,AL011955,brenda,1955,8,tropical storm,1,LA
4,4,AL011959,arlene,1959,11,tropical depression,2,LA


In [67]:
# namedStorms_df[namedStorms_df['storm_name']=='wilma']
storms_meta_fact[storms_meta_fact['storm_name'] == "emily"]

,storm_meta_id,storm_id,storm_name,storm_year,severity_id,category,location_id,storm_state_code
522,522,AL062017,emily,2017,1160,low - not tropical-subtropical-or-extratropical,123,FL
523,523,AL062017,emily,2017,1161,tropical depression,123,FL
524,524,AL062017,emily,2017,1162,tropical storm,123,FL


## storms_babyNames_fact

In [68]:
meta = storms_meta_fact[[                        
                        'storm_name',
                        'storm_year',
                        'storm_state_code',
                        'category',
                        'storm_meta_id',
                        'storm_id',
                        'location_id',
                        'severity_id'
                        ]]

bby = namesbystate_df[['babyName_id', 
                          'birth_name',
                          'birth_year',
                          'birth_state_code',
                          'popularity',
                          'gender'
                      ]]

storm_babyNames_fact = pd.merge(meta,bby, \
             left_on=['storm_name',
#                       'storm_year', 
#                       'storm_state_code'
                     ],
             right_on=['birth_name',
#                        'birth_year', 
#                        'birth_state_code'
                      ]) \
    .rename_axis('storm_babyName_id') \
    .reset_index() \
    .drop_duplicates()

storm_babyNames_fact = storm_babyNames_fact[[  
                        'storm_babyName_id',
                        'babyName_id',
                        'storm_meta_id',
                        'storm_id',
                        'storm_name',
                        'storm_year',
                        'category',
                        'storm_state_code',
                        'birth_year',
                        'birth_name',
                        'popularity',
                        'birth_state_code',
                        'gender'
                    ]]
    

storm_babyNames_fact.head()

,storm_babyName_id,babyName_id,storm_meta_id,storm_id,storm_name,storm_year,category,storm_state_code,birth_year,birth_name,popularity,birth_state_code,gender
0,0,15,0,AL011953,alice,1953,tropical depression,FL,1910,alice,410,NY,F
1,1,466,0,AL011953,alice,1953,tropical depression,FL,1911,alice,476,NY,F
2,2,929,0,AL011953,alice,1953,tropical depression,FL,1912,alice,585,NY,F
3,3,1465,0,AL011953,alice,1953,tropical depression,FL,1913,alice,603,NY,F
4,4,2026,0,AL011953,alice,1953,tropical depression,FL,1914,alice,810,NY,F


## validation

In [69]:
# ensure all storms have names 

storm_babyNames_fact[storm_babyNames_fact['storm_name']=='UNNAMED'].count()

storm_babyName_id    0
babyName_id          0
storm_meta_id        0
storm_id             0
storm_name           0
storm_year           0
category             0
storm_state_code     0
birth_year           0
birth_name           0
popularity           0
birth_state_code     0
gender               0
dtype: int64

In [70]:
# ensure the right number of metadata records follow the correct storm_id

raw = HURDAT2[HURDAT2['storm_id']=='AL172005']
useCase = namedStorms_df[namedStorms_df['storm_id']=='AL172005'].count()
print(raw, useCase)

       storm_id storm_name associated_records storm_date storm_time  \
46534  AL172005   philippe                 28        NaN        NaN   

      rec_identifier  storm_type  latitude  
46534            NaN         NaN       NaN   storm_id                    28
storm_name                  28
associated_records          28
storm_date                  28
storm_time                  28
rec_identifier              28
storm_type                  28
latitude                    28
longitude                   28
max_sustained_wind(kt)      28
minimum_pressure(mbar)      28
basin                       28
ATCF_cyclone_num_forYear    28
storm_year                  28
category                    28
coordinates                 28
storm_state_code            28
dtype: int64


In [71]:
# emily didn't make landfall in 1987, 'she' made landfall in 2017 as a tropical storm
# looking at birth names in FL in 2017 - 2019, the popularity for the name did go down

testset = storm_babyNames_fact[['storm_id', 
                                'storm_name', 
                                'storm_year', 
                                'storm_state_code',
                                'birth_year', 
                                'birth_name', 
                                'popularity', 
                                'birth_state_code', 
                                'gender']]

testset = testset[ \
         (
            (testset['birth_name']=='emily') & 
            (testset['storm_year']== 2017) &
            (testset['storm_state_code']== 'FL')
         ) & 
         ((
          (testset['birth_year']== 2017)   &  
          (testset['birth_state_code']== 'FL')
         ) |
         (
          (testset['birth_year']== 2018)   &  
          (testset['birth_state_code']== 'FL')
         ) |
          (
          (testset['birth_year']== 2021)   &  
          (testset['birth_state_code']== 'FL')
     ))
      ]

testsetProof = testset.groupby(['storm_id',
                                 'storm_name', 
                                 'storm_year', 
                                 'storm_state_code',
                                 'birth_year', 
                                 'birth_name',
                                 'birth_state_code',
                                 'gender'])\
                                ['popularity'].max()
testsetProof

storm_id  storm_name  storm_year  storm_state_code  birth_year  birth_name  birth_state_code  gender
AL062017  emily       2017        FL                2017        emily       FL                F         678
                                                    2018        emily       FL                F         579
                                                    2021        emily       FL                F         483
Name: popularity, dtype: int64

In [ ]:
namesbystate_df.count()
state_ref.count()
saffir_simpson_scale.count()
namedStorms.count()
storms_byName.count()
storms_location_dim.count()
storm_severity_dim.count()
storms_meta_fact.count()
storm_babyNames_fact.count()

# Learnings

1. Storm dataset is small and the transformations are complex
    - best to optimize the storm dataset using pandas to do the more complex transformations
    - write the optimized semi-transformed storm dataset to csv
    - do as handful of 'easy' transformations in Spark
  

2. Baby name data is large but relatively easy to transform - even in Pandas
    - EMR cluster with a single node shouldn't have any trouble ingesting the baby name data    
   
   
3. The State references table is handy.
    - best to load and transform in Spark (also counts as a second datasource, since it's json)  
  
  
4. The Saffir-Simpson reference table will need to be manually created in Spark.  


5. The schema, as designed, works in Pandas. May need to broadcast in Spark?  
    https://builtin.com/data-science/pyspark-dataframe  
    https://mungingdata.com/apache-spark/broadcast-joins/
